# Matrix operations

As linear algebra is fundamental in almost everything uses mathematics, matrices are everywhere in numerical analysis.  There isn't shortage of linear algebraic software packages and it's critically important to understand how they work.

1. POD arrays and majoring
2. Matrix-vector and matrix-matrix operations
3. Linear algebra

# POD arrays

The plain-old-data (POD) arrays are also called C-style arrays.  They are given the names because they are nothing more than just data and support no mechanism fancier than arithmetic.  We DO, oftentimes, wrap POD with fancy C++ constructs, but all the heavy-lifting numerical calculations still need to be done with POD, because essentially that's how von Neumann computers work.  (It clearly reveals itself in the machine code.  Check it yourselves.)

## Vector: 1D array

A vector is stored as a (usually contiguous) memory buffer of sequetially ordered elements.

In [1]:
!make pod_vector; ./pod_vector

make: `pod_vector' is up to date.
vector elements in memory:
  0 1 2 3 4


BLAS (basic linear albegra subprograms; a standard set of array manipulation API) defines vector operations as the 1st level.  A partial list of them:

* `SAXPY`: $\mathbf{y} = a\mathbf{x} + \mathbf{y}$, constant times a vector plus a vector
* `SDOT`: $\mathbf{x}\cdot\mathbf{y}$, dot product of two vectors.
* `SNRM2`: $\sqrt{\mathbf{y}\cdot\mathbf{y}}, $Euclidean norm.

The operations are simple enough that we usually don't need to call the library.

## Matrix: 2D array

See how to represent a $5\times5$ square matrix:

\begin{align*}
\mathrm{A} = \left[ a_{ij} \right] = \left(\begin{array}{ccccc}
a_{11} & a_{12} & a_{13} & a_{14} & a_{15} \\
a_{21} & a_{22} & a_{23} & a_{24} & a_{25} \\
a_{31} & a_{32} & a_{33} & a_{34} & a_{35} \\
a_{41} & a_{42} & a_{43} & a_{44} & a_{45} \\
a_{51} & a_{52} & a_{53} & a_{54} & a_{55}
\end{array}\right)
\end{align*}

$i$ is the row index (in the horizontal direction).  $j$ is the column index (in the vertical direction).  If we want to use the 0-based index, it can be rewritten as:

\begin{align*}
\mathrm{A} = \left[ a_{ij} \right] = \left(\begin{array}{ccccc}
a_{00} & a_{01} & a_{02} & a_{03} & a_{04} \\
a_{10} & a_{11} & a_{12} & a_{13} & a_{14} \\
a_{20} & a_{21} & a_{22} & a_{23} & a_{24} \\
a_{30} & a_{31} & a_{32} & a_{33} & a_{34} \\
a_{40} & a_{41} & a_{42} & a_{43} & a_{44}
\end{array}\right)
\end{align*}

In C++ we can use an auto variable like below for the matrix:

```cpp
constexpr size_t width = 5;

double amatrix[width][width];

// Populate the matrix on stack (row-major 2D array).
for (size_t i=0; i<width; ++i) // the i-th row
{
    for (size_t j=0; j<width; ++j) // the j-th column
    {
        amatrix[i][j] = i*10 + j;
    }
}

std::cout << "2D array elements:";
for (size_t i=0; i<width; ++i)
{
    std::cout << std::endl << " ";
    for (size_t j=0; j<width; ++j)
    {
        std::cout << " " << std::setfill('0') << std::setw(2)
                  << amatrix[i][j];
    }
}
std::cout << std::endl;
```

In [2]:
!make pod_matrix_auto; ./pod_matrix_auto

g++  -std=c++17 -O3 -g -m64 -I/opt/intel/mkl/include /opt/intel/mkl/lib/libmkl_intel_lp64.a /opt/intel/mkl/lib/libmkl_sequential.a /opt/intel/mkl/lib/libmkl_core.a -lpthread -lm -ldl  -o pod_matrix_auto pod_matrix_auto.cpp
2D array elements:
  00 01 02 03 04
  10 11 12 13 14
  20 21 22 23 24
  30 31 32 33 34
  40 41 42 43 44


The C++ multi-dimensional array index is convenient, but it doesn't work when the array size isn't known in the compile time.

```cpp
void work(double * buffer, size_t width)
{
    // This won't work as width isn't known in compile time.
    double (*matrix)[width] = reinterpret_cast<double (*)[width]>(buffer);

    for (size_t i=0; i<width; ++i) // the i-th row
    {
        for (size_t j=0; j<width; ++j) // the j-th column
        {
            matrix[i][j] = i*10 + j;
        }
    }

    std::cout << "matrix:";
    for (size_t i=0; i<width; ++i)
    {
        std::cout << std::endl << " ";
        for (size_t j=0; j<width; ++j)
        {
            std::cout << " " << std::setfill('0') << std::setw(2)
                      << matrix[i][j];
        }
    }
    std::cout << std::endl;
}
```

In [3]:
!make pod_bad_matrix

g++  -std=c++17 -O3 -g -m64 -I/opt/intel/mkl/include /opt/intel/mkl/lib/libmkl_intel_lp64.a /opt/intel/mkl/lib/libmkl_sequential.a /opt/intel/mkl/lib/libmkl_core.a -lpthread -lm -ldl  -o pod_bad_matrix pod_bad_matrix.cpp
pod_bad_matrix.cpp:7:14: error: cannot initialize a variable of type
      'double (*)[width]' with an rvalue of type 'double (*)[width]'
    double (*matrix)[width] = reinterpret_cast<double (*)[width]>(buffer);
             ^                ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
1 error generated.
make: *** [pod_bad_matrix] Error 1


## Row-major 2D variable-size array

The elements of a row-major 2D array are stored so that the fastest changing index is the trailing index of the 2D array:

\begin{align*}
\mathrm{buffer} = [a_{00}, a_{01}, a_{02}, a_{03}, a_{04}, a_{10}, a_{11}, a_{12}, \ldots, a_{43}, a_{44}]
\end{align*}

When accessing the elements, all we need to do is to remember how long we need to *stride* per row (leading) index.

```cpp
constexpr size_t width = 5;

double * buffer = new double[width*width];
double (*matrix)[width] = reinterpret_cast<double (*)[width]>(buffer);
std::cout << "buffer address: " << buffer << std::endl
          << "matrix address: " << matrix << std::endl;

// Populate a buffer (row-major 2D array).
for (size_t i=0; i<width; ++i) // the i-th row
{
    for (size_t j=0; j<width; ++j) // the j-th column
    {
        buffer[i*width + j] = i*10 + j;
    }
}

std::cout << "matrix (row-major) elements as 2D array:";
for (size_t i=0; i<width; ++i)
{
    std::cout << std::endl << " ";
    for (size_t j=0; j<width; ++j)
    {
        std::cout << " " << std::setfill('0') << std::setw(2)
                  << matrix[i][j];
    }
}
std::cout << std::endl;
```

In [4]:
!make pod_matrix_rowmajor; ./pod_matrix_rowmajor

g++  -std=c++17 -O3 -g -m64 -I/opt/intel/mkl/include /opt/intel/mkl/lib/libmkl_intel_lp64.a /opt/intel/mkl/lib/libmkl_sequential.a /opt/intel/mkl/lib/libmkl_core.a -lpthread -lm -ldl  -o pod_matrix_rowmajor pod_matrix_rowmajor.cpp
buffer address: 0x7f7f28402b90
matrix address: 0x7f7f28402b90
matrix (row-major) elements as 2D array:
  00 01 02 03 04
  10 11 12 13 14
  20 21 22 23 24
  30 31 32 33 34
  40 41 42 43 44
matrix (row-major) elements in memory:
  00 01 02 03 04 10 11 12 13 14 20 21 22 23 24 30 31 32 33 34 40 41 42 43 44
row majoring: the fastest moving index is the trailing index


## Column-major 2D variable-size array

The elements of a column-major 2D array are stored so that the fastest changing index is the leading index of the 2D array:

\begin{align*}
\mathrm{buffer} = [a_{00}, a_{10}, a_{20}, a_{30}, a_{40}, a_{01}, a_{11}, a_{21}, \ldots, a_{34}, a_{44}]
\end{align*}

Similar to a row-major array, we need to know the stride.  But this time it's for the column (trailing) index.

```cpp
constexpr size_t width = 5;

double * buffer = new double[width*width];
double (*matrix)[width] = reinterpret_cast<double (*)[width]>(buffer);
std::cout << "buffer address: " << buffer << std::endl
          << "matrix address: " << matrix << std::endl;

// Populate a buffer (column-major 2D array).
for (size_t i=0; i<width; ++i) // the i-th row
{
    for (size_t j=0; j<width; ++j) // the j-th column
    {
        buffer[j*width + i] = i*10 + j;
    }
}

std::cout << "matrix (column-major) elements as 2D array:";
for (size_t i=0; i<width; ++i)
{
    std::cout << std::endl << " ";
    for (size_t j=0; j<width; ++j)
    {
        std::cout << " " << std::setfill('0') << std::setw(2)
                  << matrix[j][i];
    }
}
std::cout << std::endl;
```

In [5]:
!make pod_matrix_colmajor; ./pod_matrix_colmajor

g++  -std=c++17 -O3 -g -m64 -I/opt/intel/mkl/include /opt/intel/mkl/lib/libmkl_intel_lp64.a /opt/intel/mkl/lib/libmkl_sequential.a /opt/intel/mkl/lib/libmkl_core.a -lpthread -lm -ldl  -o pod_matrix_colmajor pod_matrix_colmajor.cpp
buffer address: 0x7f95fe402b90
matrix address: 0x7f95fe402b90
matrix (column-major) elements as 2D array:
  00 01 02 03 04
  10 11 12 13 14
  20 21 22 23 24
  30 31 32 33 34
  40 41 42 43 44
matrix (column-major) elements in memory:
  00 10 20 30 40 01 11 21 31 41 02 12 22 32 42 03 13 23 33 43 04 14 24 34 44
column majoring: the fastest moving index is the leading index


## C++ class to treat the memory buffer like matrix

Keeping track of the stride can be error-prone.  Even if we stick to one majoring order (usually it's row-majoring), it's easy to lose track of it when the number of row and column is different, or it's higher-dimensional.

A common practice in C++ is to use a class to keep track of the stride.  Properly defined accessors significantly simplifies it.

```cpp
class Matrix {

public:

    Matrix(size_t nrow, size_t ncol)
      : m_nrow(nrow), m_ncol(ncol)
    {
        size_t nelement = nrow * ncol;
        m_buffer = new double[nelement];
    }

    // TODO: copy and move constructors and assignment operators.

    ~Matrix()
    {
        delete[] m_buffer;
    }

    // No bound check.
    double   operator() (size_t row, size_t col) const { return m_buffer[row*m_ncol + col]; }
    double & operator() (size_t row, size_t col)       { return m_buffer[row*m_ncol + col]; }

    size_t nrow() const { return m_nrow; }
    size_t ncol() const { return m_ncol; }

private:

    size_t m_nrow;
    size_t m_ncol;
    double * m_buffer;

};
```

In [6]:
!make matrix_class; ./matrix_class

g++  -std=c++17 -O3 -g -m64 -I/opt/intel/mkl/include /opt/intel/mkl/lib/libmkl_intel_lp64.a /opt/intel/mkl/lib/libmkl_sequential.a /opt/intel/mkl/lib/libmkl_core.a -lpthread -lm -ldl  -o matrix_class matrix_class.cpp
matrix:
  00 01 02 03 04
  10 11 12 13 14
  20 21 22 23 24
  30 31 32 33 34
  40 41 42 43 44


# Matrix-vector multiplication

Operations of a matrix and a vector is much more interesting than vector operations.  What we really need to do is the matrix-vector multiplication

\begin{align*}
\mathbf{y} = \mathrm{A}\mathbf{x}
\end{align*}

But because a matrix is a 2D array, we should first discuss traspose.  Write a $m\times n$ ($m$ rows and $n$ columns) matrix $\mathrm{A}$

\begin{align*}
\mathrm{A} = [a_{ij}] = \left(\begin{array}{cccc}
a_{11} & a_{12} & \cdots & a_{1n} \\
a_{21} & a_{22} & \cdots & a_{2n} \\
a_{31} & a_{32} & \cdots & a_{3n} \\
\vdots & & \ddots & \vdots \\
a_{m1} & a_{m2} & \cdots & a_{mn}
\end{array}\right)_{m\times n}
\end{align*}

its transpose $\mathrm{A}^t$ becomes a $n\times m$ ($n$ rows and $m$ columns) matrix

\begin{align*}
\mathrm{A}^t = [a_{ji}] = \left(\begin{array}{ccccc}
a_{11} & a_{21} & a_{31} & \cdots & a_{m1} \\
a_{12} & a_{22} & a_{32} & \cdots & a_{m2} \\
\vdots & & & \ddots & \vdots \\
a_{1n} & a_{2n} & a_{3n} & \cdots & a_{mn}
\end{array}\right)_{n\times m}
\end{align*}

Fast transpose can be done by taking advantage of majoring.  The key is the formula $\mathrm{A}^t = [a_{ji}]$ for $\mathrm{A} = [a_{ij}]$.  The code is like:

```cpp
double   operator() (size_t row, size_t col) const { return m_buffer[index(row, col)]; }
double & operator() (size_t row, size_t col)       { return m_buffer[index(row, col)]; }

bool is_transposed() const { return m_transpose; }

Matrix & transpose()
{
    m_transpose = !m_transpose;
    std::swap(m_nrow, m_ncol);
    return *this;
}
```

There is no data copied for transpose.  The price to pay is the if statement in the indexing helper.

```cpp
size_t index(size_t row, size_t col) const
{
    if (m_transpose) { return row          + col * m_nrow; }
    else             { return row * m_ncol + col         ; }
}
```

Come back to the matrix-vector multiplication, $\mathbf{y} = \mathrm{A}\mathbf{x}$.  The calculation is easy by using the index form of the matrix and vector.

\begin{align*}
y_i = \sum_{j=1}^n A_{ij} x_j, \quad i = 1, \ldots, m
\end{align*}

Sometimes, when Einstein's summation convention is applied, the summation sign may be suppressed, and the repeated indices imply summation

\begin{align*}
y_i = A_{ij} x_j, \quad i = 1, \ldots, m, \; j = 1, \ldots, n
\end{align*}

It can be shown that the index form of $\mathbf{y}' = \mathrm{A}^t\mathbf{x}'$ is

\begin{align*}
y'_i = A_{ji} x'_j, \quad j = 1, \ldots, m, \; i = 1, \ldots, n
\end{align*}

Implement matrix-vector multiplication:

```cpp
/*
 * Naive matrix vector multiplication.
 */
std::vector<double> operator*(Matrix const & mat, std::vector<double> const & vec)
{
    if (mat.ncol() != vec.size())
    {
        throw std::out_of_range("matrix column differs from vector size");
    }

    std::vector<double> ret(mat.nrow());

    for (size_t i=0; i<mat.nrow(); ++i)
    {
        double v = 0;
        for (size_t j=0; j<mat.ncol(); ++j)
        {
            v += mat(i,j) * vec[j];
        }
        ret[i] = v;
    }

    return ret;
}
```

In [7]:
!make matrix_vector; ./matrix_vector

g++  -std=c++17 -O3 -g -m64 -I/opt/intel/mkl/include /opt/intel/mkl/lib/libmkl_intel_lp64.a /opt/intel/mkl/lib/libmkl_sequential.a /opt/intel/mkl/lib/libmkl_core.a -lpthread -lm -ldl  -o matrix_vector matrix_vector.cpp
>>> square matrix-vector multiplication:
matrix A:
   1  0  0  0  0
   0  1  0  0  0
   0  0  1  0  0
   0  0  0  1  0
   0  0  0  0  1
vector b: 1 0 0 0 0
A*b = 1 0 0 0 0
>>> m*n matrix-vector multiplication:
matrix A:
   1  2  3
   4  5  6
vector b: 1 2 3
A*b = 14 32
>>> transposed matrix-vector multiplication:
matrix A:
   1  4
   2  5
   3  6
matrix A buffer: 1 2 3 4 5 6
vector b: 1 2
A*b = 9 12 15
>>> copied transposed matrix-vector multiplication:
matrix A:
   1  4
   2  5
   3  6
matrix A buffer: 1 4 2 5 3 6
vector b: 1 2
A*b = 9 12 15


The majoring may significantly affects the speed of matrix-vector multiplication.

# Matrix-matrix multiplication

Matrix-matrix multiplication, $\mathrm{C} = \mathrm{A}\mathrm{B}$ generally uses a $O(n^3)$ algorithm for $O(n^2)$ data.  The formula is

\begin{align*}
C_{ik} = \sum_{j=1}^n A_{ij}B_{jk}, \quad i = 1, \ldots, m, \; k = 1, \ldots, l
\end{align*}

or, by using Einstein's summation convention,

\begin{align*}
C_{ik} = A_{ij}B_{jk}, \quad i = 1, \ldots, m, \; j = 1, \ldots, n, \; k = 1, \ldots, l
\end{align*}

The C++ implementation:

```cpp
/*
 * Naive matrix matrix multiplication.
 */
Matrix operator*(Matrix const & mat1, Matrix const & mat2)
{
    if (mat1.ncol() != mat2.nrow())
    {
        throw std::out_of_range(
            "the number of first matrix column "
            "differs from that of second matrix row");
    }

    Matrix ret(mat1.nrow(), mat2.ncol());

    for (size_t i=0; i<ret.nrow(); ++i)
    {
        for (size_t k=0; k<ret.ncol(); ++k)
        {
            double v = 0;
            for (size_t j=0; j<mat1.ncol(); ++j)
            {
                v += mat1(i,j) * mat2(j,k);
            }
            ret(i,k) = v;
        }
    }

    return ret;
}
```

In [8]:
!make matrix_matrix; ./matrix_matrix

g++  -std=c++17 -O3 -g -m64 -I/opt/intel/mkl/include /opt/intel/mkl/lib/libmkl_intel_lp64.a /opt/intel/mkl/lib/libmkl_sequential.a /opt/intel/mkl/lib/libmkl_core.a -lpthread -lm -ldl  -o matrix_matrix matrix_matrix.cpp
>>> A(2x3) times B(3x2):
matrix A (2x3):
   1  2  3
   4  5  6
matrix B (3x2):
   1  2
   3  4
   5  6
result matrix C (2x2) = AB:
  22 28
  49 64
>>> B(3x2) times A(2x3):
matrix B (3x2):
   1  2
   3  4
   5  6
matrix A (2x3):
   1  2  3
   4  5  6
result matrix D (3x3) = BA:
   9 12 15
  19 26 33
  29 40 51


Matrix-matrix multiplication is intensive number-crunching.  The naive, brute-force, n-cube algorithm is basically what we need to do, without a way around.

It also demands memory.  A matrix of $100,000\times100,000$ takes $10,000,000,000$ (i.e., $10^{10}$) elements, and with double-precision floating points, it takes 80 GB.  To perform multiplication, you need the memory for 3 of the matrices, and that's 240 GB.  The dense matrix multiplication generally cannot use distributed memory without significantly loss of runtime speed.  The reasonable size of dense matrices for a workstation is around $10,000\times10,000$, i.e., 800 MB per matrix.  It's very limiting, but already facilitates a good number of applications.

# Linear algebra

After the matrix operations, it is time to introduce linear algebra in C++.  There are two critically important software packages: BLAS (http://www.netlib.org/blas/) and LAPACK (http://www.netlib.org/lapack/).  They were developed in FORTRAN.  Although the FORTRAN code is still being maintained today, it serves more like a reference implementation.  Multiple vendors provide optimized implementation, e.g., Intel's Math Kernel Library (MKL), Apple's vecLib, etc.

BLAS stands for Basic Linear Algebra Subprograms, and LAPACK is Linear Algebra PACKage.  LAPACK is designed to rely on the underneath BLAS, so the two libraries are usually used together.  For example, the general matrix-vector multiplication is defined as the `?GEMV` function in BLAS level 2 (`?` can be one of `S`, `D`, `C`, and `Z`, for single-precision real, double-precision real, single-precision complex, and double-precision complex, respectively), and the general matrix-matrix multiplication is the `?GEMM` function in BLAS level 3.

While BLAS offers basic operations like matrix multiplication, LAPACK provides more versatile computation helpers or solvers, e.g., a system of linear equations, least square, and eigen problems.

Both BLAS and LAPACK provide C interface.  They don't native C++ interface, but the C interface is compatible to C++.  CBLAS is the C interface for BLAS, and LAPACKE is that for LAPACK.

## Linear system

LAPACK provides `?GESV` functions to solve a linear system using a general (dense) matrix: $\mathrm{A}\mathbf{x} = \mathbf{b}$.  Say we have a system of linear equations:

\begin{align*}
3 x_1 + 5 x_2 + 2 x_3 &= 57 \\
2 x_1 +   x_2 + 3 x_3 &= 22 \\
4 x_1 + 3 x_2 + 2 x_3 &= 41
\end{align*}

It can be rewritten as $\mathrm{A}\mathbf{x} = \mathbf{b}$, where

\begin{align*}
\mathrm{A} = \left(\begin{array}{ccc}
3 & 5 & 2 \\
2 & 1 & 3 \\
4 & 3 & 2
\end{array}\right), \quad
\mathbf{b} = \left(\begin{array}{c}
57 \\ 22 \\ 41
\end{array}\right), \quad
\mathbf{x} = \left(\begin{array}{c}
x_1 \\ x_2 \\ x_3
\end{array}\right)
\end{align*}

Note that the reference implementation of LAPACK is FORTRAN, which uses column major.  The dimensional arguments of the LAPACK subroutines changes meaning when we call them from C with row-major matrices.

```cpp
const size_t n = 3;
int status;

std::cout << ">>> Solve Ax=b (row major)" << std::endl;
Matrix mat(n, n, std::vector<double>{3, 5, 2, 2, 1, 3, 4, 3, 2});
Matrix b = Matrix(2, n, std::vector<double>{57, 22, 41, 23, 12, 84}).transpose();
std::vector<int> ipiv(n);

std::cout << "A:" << mat << std::endl;
std::cout << "b:" << b << std::endl;

status = LAPACKE_dgesv(
    LAPACK_ROW_MAJOR // matrix_layout
  , n // n
  , b.ncol() // nrhs
  , mat.data() // a
  , mat.ncol() // lda
  , ipiv.data() // ipiv
  , b.data() // b
  , b.ncol() // ldb
  // for row major matrix, ldb becomes the trailing dimension.
);

std::cout << "solution x:" << b << std::endl;
std::cout << "dgesv status: " << status << std::endl;

std::cout << ">>> Solve Ax=b (column major)" << std::endl;
Matrix mat2 = Matrix(n, n, std::vector<double>{3, 5, 2, 2, 1, 3, 4, 3, 2}).transpose();
Matrix b2(2, n, std::vector<double>{57, 22, 41, 23, 12, 84});

std::cout << "A^t:" << mat2 << std::endl;
std::cout << "b^t:" << b2 << std::endl;

status = LAPACKE_dgesv(
    LAPACK_COL_MAJOR // matrix_layout
  , n // n
  , b2.nrow() // nrhs
  , mat2.data() // a
  , mat2.nrow() // lda
  , ipiv.data() // ipiv
  , b2.data() // b
  , b2.ncol() // ldb
  // for column major matrix, ldb remains the leading dimension.
);

std::cout << "solution x^t:" << b2 << std::endl;
std::cout << "dgesv status: " << status << std::endl;
```

In [9]:
!make gesv; ./gesv

g++  -std=c++17 -O3 -g -m64 -I/opt/intel/mkl/include /opt/intel/mkl/lib/libmkl_intel_lp64.a /opt/intel/mkl/lib/libmkl_sequential.a /opt/intel/mkl/lib/libmkl_core.a -lpthread -lm -ldl  -o gesv gesv.cpp
>>> Solve Ax=b (row major)
A:
   3  5  2
   2  1  3
   4  3  2
b:
  57 23
  22 12
  41 84
solution x:
   2 38.3913
   9 -11.3043
   3 -17.8261
dgesv status: 0
>>> Solve Ax=b (column major)
A^t:
   3  2  4
   5  1  3
   2  3  2
b^t:
  57 22 41
  23 12 84
solution x^t:
   2  9  3
  38.3913 -11.3043 -17.8261
dgesv status: 0


## Least square

LAPACK provides `?GESL` and the associated functions to find the approximated solution of an over- or under-determined system, $\min|\mathrm{A}\mathbf{x}-\mathbf{b}|$, where $\mathbf{x}$ is the unknown.

## Eigenvalue problems and SVD

LAPACK provides `?GESVD` and the associated functions for the eigenvalue problems and singular value decomposition (SVD).  Let's take SVD for example.  The problem is to the singular value and the left and right singular vector matrix

\begin{align*}
\mathrm{A}_{m\times n} = \mathrm{U}_{m\times m}\mathrm{\Sigma}_{m\times n}\mathrm{V}_{n\times n}^t
\end{align*}

where $\mathrm{U}$ is the eigenvector matrix of $\mathrm{A}\mathrm{A}^t$, $\mathrm{V}$ the eigenvector matrix of $\mathrm{A}^t\mathrm{A}$, and $\mathrm{\Sigma}$ the diagonal matrix of singular value.  Unlike an eigenvalue problem, the matrix $\mathrm{A}$ may be rectangular instead of square.

# Exercises

1. Extend the class `Matrix` to be an arbitrary dimensional array.
2. Benchmark your own matrix-vector multiplication code with BLAS DGEMV subroutine by using $1000\times1000$ or larger matrices.

# References

* BLAS: http://www.netlib.org/blas/
* LAPACK: http://www.netlib.org/lapack/